<div class="alert alert-block alert-info">
<b>Number of points for this notebook:</b> 1
<br>
<b>Deadline:</b> May 13, 2020 (Wednesday) 23:00
</div>

# Exercise 9.2. Denoising autoencoders

The goal of this exercise is to get familiar with *denoising* autoencoders.

In [ ]:
skip_training = False  # Set this flag to True before validation and submission

In [ ]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import tools
import tests

In [ ]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

In [ ]:
# Select the device for training (use GPU if you have one)
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [ ]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

# Data

In this exercise, we will use MNIST to create a new dataset (that we call varianceMNIST). In the new dataset, the information about the shapes of the digits is represented in the variances of the pixel intensities and not in the pixel intensities (like in MNIST). We use a custom `transform.Lambda()` to generate the dataset. Note that our dataset contains an infinite amount of samples because we generate different noise instances every time we request the data. The number of shapes is of course limited to the number of digits in the MNIST dataset.

This is a challenging dataset and a plain bottleneck autoencoder (from Exercise 9.1) with a mean-squared error (MSE) loss cannot encode useful information in the bottleneck layer. However, a denoising autoencoder trained with an MSE loss is able to encode the shapes of the digits in the bottleneck layer.

In [ ]:
# We will use varianceMNIST data in this exercise
transform = transforms.Compose([
    transforms.ToTensor(),  # Transform to tensor
    transforms.Lambda(lambda x: x * torch.randn_like(x))
])

trainset = torchvision.datasets.MNIST(root=data_dir, train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

We visualize some random training samples in the cell below. As you can see, we can quite easily identify the shapes and recognize the digits.

In [ ]:
images, labels = iter(trainloader).next()
tools.plot_images(images[:8], ncol=4, cmap=plt.cm.bwr, clim=[-3,3])

# Denoising autoencoder (DAE)

### Optimal denoising function

Suppose we corrupt an image $\mathbf{x}$ of the varianceMNIST dataset with a zero-mean Gaussian noise with standard deviation $\sigma_n$. For a given clean pixel value $x$, the corrupted value $\tilde{x}$ is thus produced as:
$$
\tilde{x} = x + n, \qquad n \sim \mathcal{N}(0, \sigma^2_n)
$$
Please do not confuse the corruption process with the generative process of the varianceMNIST dataset. We assume that the varianceMNIST dataset is given to us, while we are free to select any corruption process to train a DAE. In this experiment, we choose Gaussian corruption.

Knowing the generative process of the varianceMNIST dataset (which is a bit of cheating because we usually do not know the data generative process), we can compute the optimal denoising function which produces an estimate of the clean pixel value $x$ given corrupted value $\tilde{x}$:
$$
g(\tilde{x}) = \tilde{x} \: \text{sigmoid}(f(\sigma_x^2, \sigma_n^2))
$$
where $f$ is some function of the variance $\sigma^2_x$ of a pixel intensity in the varianceMNIST dataset and the variance $\sigma^2_n$ of the corruption noise.


In the cell below, your task is to implement a denoising autoencoder (DAE) which can learn to approximate the optimal denoising function shown above.
* Our DAE will be trained to learn the optimal denoising function $g(\tilde{x})$. In each training iteration, we feed corrupted images $\tilde{\mathbf{x}}$ to the inputs of the DAE and provide the corresponding clean images $\mathbf{x}$ as the targets for the DAE outputs.
* To learn useful representations (the shapes of the digits for the varianceMNIST dataset), our DAE will have a bottleneck layer with `n_components` elements. It is the output of the encoder.
* We are not going to use values of $\sigma_x^2$ and $\sigma_n^2$ inside the DAE: The value of $\sigma_x^2$ we simply do not know. We know the value of $\sigma_n^2$ (because we select the corruption process) but we are not going to use that value in the computations of the denoising function.
* Look carefully at the structure of the optimal denoising function. We can select the architecture of the DAE that makes it easy to perform the computations needed for optimal denoising.

The proposed architecture for the DAE:
* Encoder:
    * `Conv2d` layer with kernel size 5 with 6 output channels, followed by ReLU
    * `Conv2d` layer with kernel size 5 with 16 output channels, followed by ReLU
    * Fully-connected layer with 250 output features, followed by ReLU
    * Fully-connected layer with `n_components`
* Decoder:
    * Fully-connected layer with 250 output features, followed by ReLU
    * Fully-connected layer with 250 input features, followed by ReLU
    * `ConvTranspose2d` layer with kernel size 5 with 16 input channels, followed by ReLU
    * `ConvTranspose2d` layer with kernel size 5 with 6 input channels

Notes:
* The exact architecture is not tested in this notebook. The above description is not full, you need to add some missing connections using the knowledge of the form of the optimal denoising function.
* Please use recommended convolutional layers in the encoder and the decoder. If the autoencoder consists of only fully-connected layers, the learning problem is harder.

In [ ]:
class DAE(nn.Module):
    def __init__(self, n_components=10):
        # YOUR CODE HERE
        raise NotImplementedError()

    def forward(self, x):
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
def test_DAE_shapes():
    n_components = 2
    dae = DAE(n_components)

    x = torch.randn(3, 1, 28, 28)
    z, y = dae(x)
    assert z.shape == torch.Size([3, n_components]), f"Bad z.shape: {z.shape}"
    assert y.shape == x.shape, "Bad y.shape: {y.shape}"
    print('Success')

test_DAE_shapes()

# Train a denoising autoencoder

In [ ]:
# Create an autoencoder
n_components = 10
dae = DAE(n_components)
dae.to(device)

### Training loop

Implement the training loop in the cell below. Training proceeds similarly to the standard bottleneck autoencoder. The difference is that the encoder gets *corrupted* training images as inputs and the targets are the varianceMNIST digits without the corruption noise.

The recommended hyperparameters:
* Corruption of varianceMNIST images with **additive** Gaussian noise with zero mean and standard deivation $\sigma_n=0.2$.
* Adam optimizer with learning rate 0.001
* MSE loss

Hints:
- Training usually converges fast, a couple of epochs should suffice.
- The loss at convergence should be close to 0.009.

In [ ]:
# Training loop
if not skip_training:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
if not skip_training:
    tools.save_model(dae, '9_dae.pth')
else:
    dae = DAE(n_components=10)
    tools.load_model(dae, '9_dae.pth', device)

### Visualize embeddings

Let us visualize the latent space in the cell below. If your DAE does a good job, you should clearly see ten clusters corresponding to the ten classes.

In [ ]:
tests.visualize_embeddings(lambda x: dae(x)[0], trainloader, n_samples=1000, device=device)

In the cell below, we denoise some test images using the trained DAE. If your DAE does a good job, it should remove noise from the background.

In [ ]:
def plot_denoising(trainloader):
    dataiter = iter(trainloader)
    images, labels = dataiter.next()
    images = images[:4].to(device)

    with torch.no_grad():
        corrupted_images = images + 0.2 * torch.randn_like(images)
        z, reconstructions = dae(corrupted_images)
    tools.plot_images(
        torch.cat([corrupted_images, reconstructions]),
        ncol=4, cmap=plt.cm.bwr, clim=[-3,3]
    )

plot_denoising(trainloader)

## Test the quality of the produced embeddings by classification

We will test the quality of the produced encodings by training a simple linear regression classifier using the encoded images. If the classifier gives a reasonable accuracy, this is an evidence that we learned to represent the shapes of the digits in the bottleneck layer.

In [ ]:
testset = torchvision.datasets.MNIST(root=data_dir, train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

In [ ]:
# Encode data samples using the encoder
def encode(dataset, dae):
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=False)
    with torch.no_grad():
        embeddings = []
        labels = []
        for images, labels_ in dataloader:
            z, rec = dae(images.to(device))
            embeddings.append(z)
            labels.append(labels_)

        embeddings = torch.cat(embeddings, dim=0)
        labels = torch.cat(labels, dim=0)
    return embeddings, labels

traincodes, trainlabels = encode(trainset, dae)  # traincodes is (60000, 10)
testcodes, testlabels = encode(testset, dae)  # testcodes is (10000, 10)

In [ ]:
# Train a simple linear classifier
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial', max_iter=200)
logreg.fit(traincodes.cpu(), trainlabels.cpu())

predicted_labels = logreg.predict(testcodes.cpu())  # (10000,)

accuracy = np.sum(testlabels.cpu().numpy() == predicted_labels) / predicted_labels.size
print('Accuracy with a linear classifier: %.2f%%' % (accuracy*100))
assert accuracy > .83, "Poor accuracy of the embeddings: classification accuracy is %.2f%%" % (accuracy*100)
print('Success')

<div class="alert alert-block alert-info">
<b>Conclusions</b>
</div>

In this exercise, we trained a denoising autoencoder to encode meaningful information in the bottleneck layer. The codes produced in the bottleneck layer are only 10-dimensional but they can represent useful information present in the original $28 \times 28 = 784$-dimensional images. You can try to use in this task a plain bottleneck autoencoder (trained without the corruption process) with MSE loss and you will see that it fails to develop useful representations.